#### 1. Importing all the libraries

In [19]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import LineCollection

import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml

from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier


#### 2. Define the Functions

In [20]:
# Function that create the episode data - sample randomaly
def get_data(episode_size,policy,mode):
    global dataset
    if mode=='train':
        if policy==0:
             dataset=data.sample(n=episode_size)
        else:
            dataset=data
    else:
        dataset = pd.read_csv(location + '/' + file +'_test_int.csv', index_col=0)
    return dataset

In [21]:
# Function that separate the episode data into features and label
def data_separate (dataset):
    global X
    global y    
    X = dataset.iloc[:,0:dataset.shape[1]-1]  # all rows, all the features and no labels
    y = dataset.iloc[:, -1]  # all rows, label only
    return X,y

In [22]:
# Function that split the episode data into train and test
def data_split(X,y):
    global X_train_main
    global X_test_main   
    global y_train
    global y_test  
    from sklearn.model_selection import train_test_split
    X_train_main, X_test_main, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=4)
    return X_train_main, X_test_main, y_train, y_test

In [23]:
# Function that chooses exploration or explotation method
def exploration_explotation(epsilon):
    global exploration 
    if np.random.rand() < epsilon:  
        exploration=1
    else:
        exploration=0    
    return exploration

In [24]:
# Function that returns all available actions in the state given as an argument: 
def available_actions(number_of_columns,columns,initial_state,current_state,trashold, exploration):
    global exclude
    global all_columns
#    exclude=[]
    all_columns=np.arange(number_of_columns+1)
    # remove columns that have been already selected
    exclude=columns.copy()
    # remove the initial_state and the current_state
    exclude.extend([initial_state, current_state])
    available_act = list(set(all_columns)-set(exclude))
    # remove actions that have negetiv Q value
    if exploration==0:
        index = np.where(Q[current_state,available_act] > trashold)[1]
        available_act= [available_act[i] for i in index.tolist()]
    return available_act

In [25]:
def sample_next_action(current_state, Q, available_act, exploration):
    global available_act_q_value
    available_act_q_value = [float(q) for q in np.array(Q[current_state, available_act]).reshape(-1)]
    
    if exploration == 1: 
        # Random selection
        next_action = int(np.random.choice(available_act, 1).item())
    else: 
        # Greedy selection according to max value
        maxQ = max(available_act_q_value)
        count = available_act_q_value.count(maxQ)
        
        if count > 1:
            max_columns = [i for i in range(len(available_act_q_value)) if available_act_q_value[i] == maxQ]
            i = int(np.random.choice(max_columns, 1).item())
        else:
            i = available_act_q_value.index(maxQ)
        
        next_action = available_act[i]  
    
    return next_action


In [26]:
# function that update a list with all selected columns in the episode
def update_columns(action, columns):
    update_columns=columns
    update_columns.append(action)
    return update_columns

In [27]:
# function that update the X_train and X_test according to the current episode columns list 
def update_X_train_X_test(columns,X_train_main, X_test_main):
    X_train=X_train_main.iloc[:,columns]
    X_test=X_test_main.iloc[:,columns]
    X_train=pd.DataFrame(X_train)
    X_test=pd.DataFrame(X_test)
    return X_train, X_test

In [28]:
# Function that run the learner and get the error to the current episode columns list
def Learner(X_train, X_test,y_train, y_test):
    global learner
    global y_pred
    if learner_model == 'DT':
        learner = tree.DecisionTreeClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)
    elif learner_model == 'KNN':
        learner = KNeighborsClassifier(metric='hamming',n_neighbors=5)
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'SVM':
        learner = SVC()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)        
    elif learner_model == 'NB':
        learner = MultinomialNB()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)
    elif learner_model == 'AB':
        learner = AdaBoostClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    elif learner_model == 'GB':
        learner = GradientBoostingClassifier()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    elif learner_model == 'VQC':
        learner = QuantumLearner()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    elif learner_model == 'ANN':
        learner = ClassicalLearner()
        learner = learner.fit(X_train, y_train)
        y_pred = learner.predict(X_test)  
    accuracy=metrics.accuracy_score(y_test, y_pred)
    error=1-accuracy
    return error


In [29]:
def q_update(current_state, action, learning_rate, reward):
    # next_state = current action
    max_index = np.where(Q[action,] == np.max(Q[action,]))[0]  # Use [0] instead of [1] for 1D arrays
    
    if max_index.shape[0] > 1:
        # Resolve tie by selecting one randomly
        max_index = int(np.random.choice(max_index, size=1).item())
    else:
        max_index = int(max_index[0])  # Convert the first element to a scalar

    max_value = Q[action, max_index]

    # Update the Q matrix
    if Q[current_state, action] == 1:
        Q[current_state, action] = learning_rate * reward
    else:
        Q[current_state, action] = Q[current_state, action] + learning_rate * (
            reward + (discount_factor * max_value) - Q[current_state, action]
        )


### Experiment mangment

#### 3. Define the parameters 

In [34]:
## for run time ##
N_features=2
N_data=1
## for run time ##

#Experiment: 
experiment='test'
number_of_experiment=1

# Dataset parameters #
location = 'Datasets/adult'
outputlocation='Datasets'
file='adult' #adult #diabetic_data #no_show
#np.random.seed(3)

# Q learning parameter # 
learning_rate=0.005
discount_factor = 0.01 #0
epsilon = 0.1

# Learner and episode parameters #
learner_model = 'VQC' #DT #KNN #SVM
episode_size=100
internal_trashold=0
external_trashold=0
filename= file +'_int.csv'

#Experiments folder management: 
#if not os.path.exists('/Experiments'):
#    os.makedirs('/Experiments') 
if not os.path.exists('Experiments/'+ str(experiment)):
    os.makedirs('Experiments/'+ str(experiment))
else:
    shutil.rmtree('Experiments/'+ str(experiment))          #removes all the subdirectories!
    os.makedirs('Experiments/'+ str(experiment))
#writer = pd.ExcelWriter('Experiments/'+ str(experiment) + '/df.xlsx') 



text_file = open('Experiments/'+ str(experiment) +'/parameters.txt', "w")
text_file.write('experiment: ' + str(experiment)+ '\n')
text_file.write('number of experiments: ' + str(number_of_experiment)+ '\n')
text_file.write('file: ' + str(file)+ '\n')
text_file.write('learner model: ' + str(learner_model)+ '\n')
text_file.write('episode size: ' + str(episode_size)+ '\n')
#text_file.write('numbers of epocs: ' + str(epocs)+ '\n')
text_file.write('internal trashold: ' + str(internal_trashold)+ '\n')
text_file.write('external trashold: ' + str(external_trashold)+ '\n')
 
text_file.close()

#### 4. Run all experiments

In [35]:
for e in range (number_of_experiment):
    if not os.path.exists('Experiments/'+ str(experiment)+ '/'+ str(e)):
        os.makedirs('Experiments/'+ str(experiment)+ '/'+ str(e))
    else:
        shutil.rmtree('Experiments/'+ str(experiment)+ '/'+ str(e))          #removes all the subdirectories!
        os.makedirs('Experiments/'+ str(experiment)+ '/'+ str(e))
    print ('Experiments ' + str(e) + ' start')
##########################Experiment setup##########################
    # Read the data
    data = pd.read_csv(location + '/' + filename, index_col=0)
    
##### for run time - start #####
    import timeit
    start = timeit.default_timer()
    size= int(N_data* len(data.index))
    data = data.sample(n=size)
    data=data.iloc[:,-N_features-1:]
##### for run time - end #####
    
    #Set the number of iterations:
    interations=10*len(data.index)/episode_size
    # Set the number of columns exclude the class column
    number_of_columns=data.shape[1]-1 
    print ("number of columns: "+ str(number_of_columns) +" (exclude class column)" ) 
    # Set the number of episodes 
    # episodes_number=epocs*len(data.index)/episode_size
    episodes_number=interations
    print ("Number of episodes: "+ str(episodes_number) ) 
    # Initialize matrix Q as a 1 values matrix:
    #Q = np.matrix(np.ones([number_of_columns+1,number_of_columns+1])) # we will use the last dummy columns as initial state s
    Q = np.matrix(np.ones([number_of_columns+1,number_of_columns+1])) # we will use the last dummy columns as initial state s
    # Set initial_state to be the last dummy column we have created
    initial_state=number_of_columns
    # define data frame to save episode policies results
    df = pd.DataFrame(columns=('episode','episode_columns','policy_columns','policy_accuracy_train','policy_accuracy_test'))
    print ("initial state number: "+ str(initial_state) + " (the last dummy column we have created)") 

    ##########################  episode  ##########################  
    for i in range (int(episodes_number)):
    ########## Begining of episode  ############
        # Initiate lists for available_act, episode_columns and and the policy mode & episode_error
        episode_available_act=list(np.arange(number_of_columns))
        episode_columns=[]
        policy=0
        episode_error=0
        # Initiate the error to 0.5
        episode_last_error=0.5
        # Initiate current_state to be initial_state
        episode_current_state=initial_state
        # Create the episode data 
        episode= get_data(episode_size, policy=0, mode='train')
        # Separate the episode data into features and label
        X_episode,y_episode=data_separate(episode)
        # Split the data into train and test 
        X_train_main_episode, X_test_main_episode, y_train_episode, y_test_episode = data_split(X_episode,y_episode)
        if i<episodes_number*0.25:
            epsilon=0.9
            learning_rate=0.09
        elif i<episodes_number*0.5:
            epsilon=0.5
            learning_rate=0.05
        elif i<episodes_number*0.75:
            epsilon=0.3
            learning_rate=0.01
        else:
            epsilon=0.1
            learning_rate=0.005
        ########## Q learning start ############

        while len(episode_available_act)>0:
            # Get exploration or explotation flag 
            exploration=exploration_explotation(epsilon)
            # Get available actions in the current state
            episode_available_act = available_actions(number_of_columns,episode_columns,initial_state,episode_current_state,internal_trashold,exploration)
            if len(episode_available_act)>0:
                # Sample next action to be performed
                episode_action = sample_next_action(episode_current_state, Q, episode_available_act, exploration)
                # Update the episode_columns
                episode_columns=update_columns(episode_action,episode_columns)
                # Update the dataset to include all episode columns + current selected action (column)
                X_train_episode, X_test_episode =update_X_train_X_test(episode_columns,X_train_main_episode, X_test_main_episode)
                # Update the accuracy of the current columns
                episode_error= Learner(X_train_episode, X_test_episode, y_train_episode, y_test_episode)
                # Update reward
                episode_reward=episode_last_error-episode_error
                # Update Q matrix
                q_update(episode_current_state,episode_action,learning_rate, episode_reward)
                # Update parameters for next round 
#                 if episode_current_state==initial_state:
#                     beta=abs(episode_reward-Q[episode_current_state,episode_action])
#                     epsilon=final_epsilon+(beta*(1-final_epsilon))
                #    learning_rate=final_learning_rate+(beta*(1-final_learning_rate))
                episode_current_state=episode_action
                episode_last_error=episode_error
                 
        ########## Q learning End ############

        #Save Q matrix: 
        if (i%100 ==0):
            Q_save=pd.DataFrame(Q)
            Q_save.to_csv('Experiments/'+ str(experiment)+ '/'+ str(e)+ '/Q.'+ str(i+1) + '.csv') 

        # Calculate policy 
        policy_available_actions=list(np.arange(number_of_columns))
        policy_columns=[]
        policy_current_state=initial_state
        while len(policy_available_actions)>0:
            # Get available actions in the current state
            policy_available_actions = available_actions(number_of_columns,policy_columns,initial_state,policy_current_state, external_trashold, exploration=0)
            # # Sample next action to be performed
            if len(policy_available_actions)>0:
                policy_select_action = sample_next_action(policy_current_state, Q, policy_available_actions, exploration=0)
                # Update the episode_columns
                policy_columns=update_columns(policy_select_action,policy_columns)
                policy_current_state=policy_select_action
        # Calculate policy_accuracy    
        if len(policy_columns)>0:
            ##for training dataset##
            policy_data=get_data(episode_size,policy=1,mode='train')
            X_policy,y_policy=data_separate(policy_data)
            X_train_main_policy, X_test_main_policy, y_train_policy, y_test_policy = data_split(X,y)
            X_train_policy, X_test_policy =update_X_train_X_test(policy_columns, X_train_main_policy, X_test_main_policy)
            policy_error=Learner(X_train_policy, X_test_policy,y_train_policy, y_test_policy)
            policy_accuracy_train=1-policy_error
            ##for testing dataset##
            policy_data=get_data(episode_size,policy=1,mode='test')
            X_policy,y_policy=data_separate(policy_data)
            X_train_main_policy, X_test_main_policy, y_train_policy, y_test_policy = data_split(X,y)
            X_train_policy, X_test_policy =update_X_train_X_test(policy_columns, X_train_main_policy, X_test_main_policy)
            policy_error=Learner(X_train_policy, X_test_policy,y_train_policy, y_test_policy)
            policy_accuracy_test=1-policy_error 
        else:
            policy_accuracy_train=0 
            policy_accuracy_test=0
        #df=df.append({'episode':str(i+1), 'episode_columns':str(episode_columns),'policy_columns':str(policy_columns),'policy_accuracy_train':policy_accuracy_train,'policy_accuracy_test':policy_accuracy_test}, ignore_index=True)
        #new_row = pd.DataFrame([{'episode': str(i+1),
        #                  'episode_columns': str(episode_columns),
        #                  'policy_columns': str(policy_columns),
        #                  'policy_accuracy_train': policy_accuracy_train,
        #                  'policy_accuracy_test': policy_accuracy_test}])
        #df = pd.concat([df, new_row], ignore_index=True)
        df.loc[len(df)] = {
            'episode': str(i+1),
            'episode_columns': str(episode_columns),
            'policy_columns': str(policy_columns),
            'policy_accuracy_train': policy_accuracy_train,
            'policy_accuracy_test': policy_accuracy_test
        }

        #Prints
        print ("episode "+ str(i+1) +" start") 
        print ("episode columns: "+ str(episode_columns) + " epsilon: " + str(epsilon) + " learning rate: " + str(learning_rate) + " error: " +str(episode_error))
        print ("episode policy:" + str(policy_columns) + " train accuracy: " + str(policy_accuracy_train)  + " test accuracy: " +str(policy_accuracy_test)) 
        print ("episode "+ str(i+1) +" end") 
    ########## End of episode  ############
    #df.to_excel(writer, 'Experiment' + str(e))
    df.to_excel(writer, sheet_name='Experiment' + str(e))
    df_plot=df[['episode','policy_accuracy_train','policy_accuracy_test']]
    plot=df_plot.plot()
    fig = plot.get_figure()
    fig.savefig('Experiments/'+ str(experiment) + '/plot_experiment_' + str(e) +'.png')
    
#writer.save()
with pd.ExcelWriter('Experiments/'+ str(experiment) + '/df.xlsx') as writer:
    df.to_excel(writer, sheet_name='Experiment' + str(e))

## for run time ##
stop = timeit.default_timer()
print (stop - start)
## for run time ##

Experiments 0 start
number of columns: 2 (exclude class column)
Number of episodes: 2443.0
initial state number: 2 (the last dummy column we have created)



KeyboardInterrupt



In [ ]:
# Quantum Learner based on Benedetti et al.
class QuantumLearner:
    def __init__(self, num_layers=2):
        self.num_layers = num_layers
        self.weights = None
        self.bias = None
        self.num_qubits = None
        self.dev = None
        self.opt = qml.optimize.AdamOptimizer(0.05)

    def _initialize_circuit(self, num_features):
        # Update the number of qubits to match the current feature count.
        self.num_qubits = num_features
        self.dev = qml.device('qiskit.aer', wires=list(range(self.num_qubits)))
        #self.dev = qml.device("default.qubit", wires=list(range(self.num_qubits)))

        @qml.qnode(self.dev, interface="autograd")
        def circuit(weights, x):
            self.feature_encoding(x)
            for W in weights:
                self.variational_layer(W)
            return qml.expval(qml.PauliZ(0))
        
        self.circuit = circuit

    def feature_encoding(self, x):
        for i in range(self.num_qubits):
            qml.RY(np.pi * x[i], wires=i)
        for i in range(self.num_qubits - 1):
            qml.CZ(wires=[i, i + 1])

    def variational_layer(self, W):
        for i in range(self.num_qubits):
            qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
        for i in range(self.num_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        if self.num_qubits > 1:
            qml.CNOT(wires=[self.num_qubits - 1, 0])

    def variational_classifier(self, weights, bias, x):
        return self.circuit(weights, x) + bias

    def cost(self, weights, bias, X, Y):
        # X is expected to be a NumPy array.
        predictions = qml.numpy.array([self.variational_classifier(weights, bias, x) for x in X])
        return qml.numpy.mean((qml.numpy.array(Y) - predictions) ** 2)

    def fit(self, X_train, y_train, num_it=100, batch_size=48, warm_start=False):
        # Convert inputs to NumPy arrays if needed.
        if hasattr(X_train, "values"):
            X_train = X_train.values.astype(np.float64)
        else:
            X_train = np.array(X_train, dtype=np.float64)
        if hasattr(y_train, "values"):
            y_train = y_train.values
        else:
            y_train = np.array(y_train)
        
        current_features = X_train.shape[1]
        
        # If not warm starting or if the number of features has changed, reinitialize the circuit and parameters.
        if not warm_start or (self.num_qubits is None) or (self.num_qubits != current_features):
            self._initialize_circuit(current_features)
            np.random.seed(0)
            self.weights = qml.numpy.tensor(
                0.01 * np.random.randn(self.num_layers, self.num_qubits, 3),
                requires_grad=True,
            )
            self.bias = qml.numpy.tensor(0.0, requires_grad=True)

        batch_size = min(batch_size, len(X_train))
        for it in range(num_it):
            batch_index = np.random.choice(len(X_train), batch_size, replace=False)
            X_batch = X_train[batch_index]
            Y_batch = y_train[batch_index]
            self.weights, self.bias = self.opt.step(
                lambda w, b: self.cost(w, b, X_batch, Y_batch), self.weights, self.bias
            )
        return self

    def predict(self, X_test):
        if hasattr(X_test, "values"):
            X_test = X_test.values.astype(np.float64)
        else:
            X_test = np.array(X_test, dtype=np.float64)
        return np.array([
            float(qml.numpy.sign(self.variational_classifier(self.weights, self.bias, x)))
            for x in X_test
        ])


In [17]:
# Classical Learner based on a simple ANN
class ClassicalLearner(nn.Module):
    def __init__(self, num_layers=2, hidden_size=5):
        super().__init__()
        self.layers = None
        self.sigmoid = nn.Sigmoid()

    def initialize_layers(self, input_size, num_layers, hidden_size=5):
        layers = [input_size] + [hidden_size] * (num_layers - 1) + [1]
        self.layers = nn.ModuleList([nn.Linear(layers[i], layers[i+1], dtype=torch.float64) for i in range(len(layers) - 1)])
    
    def forward(self, x):
        x = x.to(torch.float64)
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        return self.sigmoid(self.layers[-1](x))
    
    def fit(self, X_train, y_train, num_it=50, lr=0.01):
        input_size = X_train.shape[1]
        self.initialize_layers(input_size, num_layers=2)
        optimizer = optim.Adam(self.parameters(), lr=lr)
        y_train = torch.tensor(y_train.values, dtype=torch.float64).reshape(-1, 1)
        
        for epoch in range(num_it):
            optimizer.zero_grad()
            y_pred = self.forward(torch.tensor(X_train.values, dtype=torch.float64)).reshape(-1, 1)
            loss = nn.BCELoss()(y_pred, y_train)
            loss.backward()
            optimizer.step()
        
        return self
    
    def predict(self, X_test):
        with torch.no_grad():
            X_test_tensor = torch.tensor(X_test.values, dtype=torch.float64)
            y_pred = self.forward(X_test_tensor).reshape(-1, 1)
            return (y_pred.numpy().flatten() > 0.5).astype(int)


In [18]:
# Quantum Learner based on Benedetti et al.
class QuantumLearner:
    def __init__(self, num_layers=2):
        self.num_layers = num_layers
        self.weights = None
        self.bias = None
        self.num_qubits = None
        self.dev = None
        self.opt = qml.optimize.AdamOptimizer(0.05)

    def _initialize_circuit(self, num_features):
        # Update number of qubits (and rebuild device) to match the current feature count.
        self.num_qubits = num_features
        self.dev = qml.device("default.qubit", wires=list(range(self.num_qubits)))

        @qml.qnode(self.dev, interface="autograd")
        def circuit(weights, x):
            self.feature_encoding(x)
            for W in weights:
                self.variational_layer(W)
            return qml.expval(qml.PauliZ(0))
        
        self.circuit = circuit

    def feature_encoding(self, x):
        # Encode features using RY rotations and entangle adjacent qubits with CZ.
        for i in range(self.num_qubits):
            qml.RY(np.pi * x[i], wires=i)
        for i in range(self.num_qubits - 1):
            qml.CZ(wires=[i, i + 1])

    def variational_layer(self, W):
        # Apply a parameterized rotation on each qubit.
        for i in range(self.num_qubits):
            qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
        # Apply CNOT gates between adjacent qubits.
        for i in range(self.num_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        # If there is more than one qubit, add a wrap‑around CNOT.
        if self.num_qubits > 1:
            qml.CNOT(wires=[self.num_qubits - 1, 0])

    def variational_classifier(self, weights, bias, x):
        # Compute the circuit’s output and add a bias.
        return self.circuit(weights, x) + bias

    def cost(self, weights, bias, X, Y):
        # Compute mean squared error cost over the entire training set.
        predictions = qml.numpy.array([self.variational_classifier(weights, bias, x) for x in X])
        return qml.numpy.mean((qml.numpy.array(Y) - predictions) ** 2)

    def fit(self, X_train, y_train, num_it=50, warm_start=False, verbose=False):
        """
        Fit the quantum learner.
        
        Parameters:
          - X_train, y_train: training data (can be pandas DataFrame/Series or NumPy arrays)
          - num_it: number of epochs (full forward/backward passes over the full training set)
          - warm_start: if True, retain parameters if the feature set (number of qubits) is unchanged.
          - verbose: if True, print cost and training accuracy each epoch.
        """
        # Convert inputs to NumPy arrays if needed.
        if hasattr(X_train, "values"):
            X_train = X_train.values.astype(np.float64)
        else:
            X_train = np.array(X_train, dtype=np.float64)
        if hasattr(y_train, "values"):
            y_train = y_train.values
        else:
            y_train = np.array(y_train)
        
        current_features = X_train.shape[1]
        # If not warm starting, or if the number of features has changed, reinitialize.
        if (not warm_start) or (self.num_qubits is None) or (self.num_qubits != current_features):
            self._initialize_circuit(current_features)
            np.random.seed(0)
            self.weights = qml.numpy.tensor(
                0.01 * np.random.randn(self.num_layers, self.num_qubits, 3),
                requires_grad=True,
            )
            self.bias = qml.numpy.tensor(0.0, requires_grad=True)
        
        # Train over multiple epochs using the full training set each epoch.
        for epoch in range(num_it):
            # Perform one optimization step on the entire training set.
            self.weights, self.bias = self.opt.step(
                lambda w, b: self.cost(w, b, X_train, y_train),
                self.weights, self.bias
            )
            if verbose:
                cost_val = self.cost(self.weights, self.bias, X_train, y_train)
                # Compute training accuracy by thresholding the circuit output.
                predictions = qml.numpy.array([
                    qml.numpy.sign(self.variational_classifier(self.weights, self.bias, x))
                    for x in X_train
                ])
                # Note: assuming y_train labels are in {-1, 1}.
                acc = qml.numpy.mean(qml.numpy.abs(qml.numpy.array(y_train) - predictions) < 1e-5)
                print(f"Epoch: {epoch+1:3d} | Cost: {cost_val:0.7f} | Accuracy: {acc:0.7f}")
        
        return self

    def predict(self, X_test):
        # Convert to NumPy array if needed.
        if hasattr(X_test, "values"):
            X_test = X_test.values.astype(np.float64)
        else:
            X_test = np.array(X_test, dtype=np.float64)
        return np.array([
            float(qml.numpy.sign(self.variational_classifier(self.weights, self.bias, x)))
            for x in X_test
        ])
